In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 29.1 MB/s eta 0:00:00


In [ ]:
import pickle
import tensorflow as tf
from keras.utils import pad_sequences
from transformers import AutoTokenizer
import numpy as np

class TargetedHSD:
    def __init__(self, model_path = None, tokenizer_path = None):
        if not model_path:
            self.__model_path = 'drive/MyDrive/CODE/ViTHSD/model/xlm-roberta-base.h5'
        else:
            self.__model_path = model_path
        if not tokenizer_path:
            self.__tokenizer_path = 'xlm-roberta-base'
        else:
            self.__tokenizer_path = tokenizer_path
        
        self._tokenizer = AutoTokenizer.from_pretrained(self.__tokenizer_path)

        self.result = None
        self.orginal_label = None
    
    def predict(self, text):
        # encoded_text = self._tokenizer.texts_to_sequences([text])
        # encoded_text = pad_sequences(encoded_text, maxlen=100, padding='post')
        encoded_text = np.array(self._tokenizer([text], max_length=100, padding='max_length', truncation=True)['input_ids'])
        # pred = self._model.predict(encoded_text)
        pred = np.argmax(encoded_text.reshape(-1, 5, 4), axis=-1)

        # y_test_pred_new = []
        # for y in pred:
        #     lb = []
        #     for i in range(0, len(y)):
        #         if y[i] >= 0.5:
        #             lb.append(1)
        #         else:
        #             lb.append(0)
        #     y_test_pred_new.append(lb)
        self.orginal_label = pred[0]
    
    def return_label(self):
        true_labels = []
        TYPE = {
            1: "clean",
            2: "offensive",
            3: "hate"
        }
        LABEL = {
            0: "individual",
            1: "groups",
            2: "religion",
            3: "race",
            4: "politics"
        }
        # LABEL = [('individual', 1),
        #             ('individual', 2),
        #             ('individual', 3),
        #             ('groups', 1),
        #             ('groups', 2),
        #             ('groups', 3),
        #             ('religion/creed', 1),
        #             ('religion/creed', 2),
        #             ('religion/creed', 3),
        #             ('race/ethnicity', 1),
        #             ('race/ethnicity', 2),
        #             ('race/ethnicity', 3),
        #             ('politics', 1),
        #             ('politics', 2),
        #             ('politics', 3)
        #         ]
        print(self.orginal_label)
        for i in range(0, len(self.orginal_label)):
            if self.orginal_label[i] > 0:
                t = LABEL[i] + "#" + TYPE[int(self.orginal_label[i])]
                true_labels.append(t)

        self.result = true_labels
        return true_labels 

In [73]:
cls = TargetedHSD()

In [155]:
# Nhap vao day
text = "chào bạn nhé"

In [156]:
cls.predict(text)

In [157]:
cls.return_label()

[1 0 0 0 0]


['individual#clean']